In [2]:
import os
import re
import csv

def clean_content(text):
    """기사 본문에서 불필요한 출처 및 특수문자 제거"""
    text = re.sub(r"\[.*?\]", "", text)  # 대괄호 [] 안의 내용 제거
    text = re.sub(r"\(.*?\)", "", text)  # 소괄호 () 안의 내용 제거
    text = re.sub(r"(\w+ 기자|\w+ 특파원)", "", text)  # '기자', '특파원' 포함된 단어 제거
    text = re.sub(r"[^\w\s\.\?!]", "", text)  # 특수문자만 제거, 구두점은 살려두고
    text = re.sub(r"\d+", "", text)  # 숫자 제거
    text = re.sub(r"\s+", " ", text)  # 연속된 공백 제거
     # 저작권 문구 제거
    text = re.sub(r"저작권자 , 무단 , 전재재배포 ,  파이낸셜뉴스 , 금지", "", text)
    text = re.sub(r"저작권자 . 무단 전재재배포 금지", "", text)
    text = re.sub(r"본 자료는 해당 업체에서 제공한 보도 자료입니다.", "", text)
    text = re.sub(r"알기쉬운 채권투자김형철", "", text)
    text = re.sub(r"저작권자 파이낸셜뉴스. 무단 전재재배포 금지", "", text)
    text = re.sub(r"공식 SNS계정 파이낸셜뉴스 패밀리사이트", "", text)
    text = re.sub(r"파이낸셜뉴스 패밀리사이트 공식 SNS계정", "", text)
    text = re.sub(r"파이낸셜뉴스 실리콘밸리", "", text)
    text = re.sub(r"파이낸셜뉴스 도쿄", "", text)
    text = re.sub(r"전국투자자교육협의회http", "", text)
    text = re.sub(r"암진단시 배 연금지급", "", text)

    # 문장을 . 기준으로 나누고, '헉!'이 포함된 문장 삭제
    sentences = text.split('.')
    sentences = [sentence.strip() for sentence in sentences if '헉!' not in sentence]
    text = '. '.join(sentences)

    return text.strip()

def extract_date_and_text(text):
    """날짜와 본문 내용 추출"""
    pattern = r'(\d{4}-\d{2}-\d{2})(.*?)(?=/$|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    
    extracted_data = []
    for match in matches:
        date, content = match
        # '/' 뒤의 내용 제거
        content = content.split('/')[0].strip()
        
        # 텍스트 클린징 (불필요한 부분 제거 후 클린징)
        content = clean_content(content)
        
        extracted_data.append((date, content))
    
    return extracted_data

def process_files_in_directory(directory_path, output_file):
    """디렉터리 내 .txt 파일을 처리하여 CSV로 저장"""
    header = ['date', 'title', 'id', 'content']
    
    with open(output_file, 'w', encoding='utf-8', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(header)  # 헤더 작성
        
        for filename in os.listdir(directory_path):
            if filename.endswith('.txt'):  # .txt 파일만 처리
                file_path = os.path.join(directory_path, filename)
                
                # 파일명에서 id 추출 (news_YYYY-MM-DD_0000000000.txt)
                file_id = filename.split('_')[-1].replace('.txt', '')
                
                with open(file_path, 'r', encoding='utf-8') as in_file:
                    text = in_file.read()
                    
                    # 텍스트 첫 번째 행을 title로 추출
                    title = text.split('\n')[0].strip()
                    
                    # 날짜와 본문 추출
                    extracted_data = extract_date_and_text(text)
                    
                    # 추출한 내용을 CSV 파일에 기록
                    for date, content in extracted_data:
                        writer.writerow([date, title, file_id, content])

# 폴더 경로와 출력 CSV 파일 경로 지정
directory_path = 'C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/뉴스 스크래피 원본자료/news_texts'  # 폴더 경로를 여기에 넣으세요
output_file = '파이낸셜 최종 클렌징.csv'  # 결과를 저장할 CSV 파일 이름

# 파일 처리 함수 호출
process_files_in_directory(directory_path, output_file)
